*In this notebook, 

# Indicators description

### Stability Score Formula

The stability score is calculated using a weighted combination of key features:

\begin{equation}
\text{Stability Score} = 
W_1 \times \text{Day Consistency Score} +
W_2 \times \text{Evening Consistency Score} +
W_3 \times \text{Night Consistency Score} +
W_4 \times \text{Unique Hours} +
W_5 \times \text{Max Consecutive Hours} +
W_6 \times \log(\text{Total Pings}) +
W_7 \times \text{Consistency Score} +
W_8 \times \text{Dominance Score} +
W_9 \times \text{Time Window Coverage} +
W_{10} \times \text{Weekend Focus Score} +
W_{11} \times \text{Entropy}
\end{equation}

---

### Metric Definitions

- **$W_1$–$W_3$**: Weights for consistency during **day**, **evening**, and **night** periods (based on presence during those time windows across the quarter).
- **$W_4$**: Weight for **Unique Hours** — favors clusters with broader hourly presence.
- **$W_5$**: Weight for **Max Consecutive Hours** — captures un
interrupted stays (also capturing silent hours)
- **$W_6$**: Weight for **Log Total Pings** — log-scaled to reward frequent clusters while reducing the impact of outliers.
- **$W_7$**: Weight for **Consistency Score** — measures how often a cluster appears across the quarter relative to total days with any activity.
- **$W_8$**: Weight for **Dominance Score**, the proportion of all device pings in the quarter that occurred in this cluster.
- **$W_9$**: Weight for **Time Window Coverage**, computed as:

\begin{equation}
\text{Time Window Coverage} = \frac{\mathbb{1}_{\text{day}} + \mathbb{1}_{\text{evening}} + \mathbb{1}_{\text{night}}}{3}
\end{equation}

where $\mathbb{1}_{\text{window}} = 1$ if the cluster was active in that time window.

- **$W_{10}$**: Weight for **Weekend Focus Score**, which favors clusters that are more active on weekends — a strong indicator of home-like behavior. It is calculated as:

\begin{equation}
\text{Weekend Focus Score} = \frac{D_{\text{weekend}}}{D_{\text{weekend}} + D_{\text{weekday}} + \epsilon}
\end{equation}

where $D_{\text{weekend}}$ and $D_{\text{weekday}}$ are the number of unique weekend and weekday days the cluster was active, and $\epsilon$ is a small value to avoid division by zero.


- **$W_{11}$** :  Weight for Entropy Score, which captures how evenly the pings are distributed across the hours of the day. Lower entropy indicates a more focused, potentially stable behavior (e.g., always pinging at night), while higher entropy indicates scattered presence.

---

### Consistency Score

The **Consistency Score** captures how persistently a cluster appears across the quarter:

\begin{equation}
\text{Consistency Score}_{c,q}^{(i)} = \frac{\text{Days Cluster Active}_{c,q}^{(i)}}{\text{Days Device Active}_{q}^{(i)}}
\end{equation}

- $c$ = cluster index  
- $q$ = quarter  
- $i$ = unique device identifier (CAID)  
- $\text{Days Cluster Active}_{c,q}^{(i)}$ = number of unique days the cluster was observed  
- $\text{Days Device Active}_{q}^{(i)}$ = total days with any ping for the CAID

---

### Why Log-Scale Total Pings?

Log-scaling reduces the influence of outliers with extremely high ping counts, while still rewarding highly visited clusters. This ensures stability is not biased toward sheer volume alone.




# Import data

In [58]:
import pandas as pd
final_data = pd.read_csv('../datasets/clustered_data.csv')

# Create hour column and period columns (day, evening, night)

In [59]:
"""
We create columns is_day, is_evening, is_night, and hour from the datetime column.
is_day is 1 between 8am (included) and 8pm (not included), otherwise 0.
is_evening is 1 between 8pm (included) and 12pm (not included), otherwise 0.
is_night is 1 between 12pm (included) and 8am (not included)

"""


import pandas as pd
import numpy as np
from datetime import datetime

# Convert timestamps to datetime using a flexible approach
final_data['datetime'] = pd.to_datetime(final_data['utc_timestamp'], format='mixed')

# Add time period indicators
def time_period(timestamp):
    hour = timestamp.hour
    if 8 <= hour < 20:
        return 'day'
    elif 20 <= hour < 24:
        return 'evening'
    else:  # 12 AM to 8 AM
        return 'night'

final_data['time_period'] = final_data['datetime'].apply(time_period)

# Create indicator columns for each period
final_data['is_day'] = (final_data['time_period'] == 'day').astype(int)
final_data['is_evening'] = (final_data['time_period'] == 'evening').astype(int)
final_data['is_night'] = (final_data['time_period'] == 'night').astype(int)

# Add date column for consistency score
final_data['date'] = final_data['datetime'].dt.date
# Make sure date column is datetime type
final_data['date'] = pd.to_datetime(final_data['date'])
# Drop the original 'time_period' column
final_data.drop('time_period', axis=1, inplace=True)

# Extract hour from datetime
final_data['hour'] = final_data['datetime'].dt.hour


In [60]:
final_data.head()

,caid,zipcode,utc_timestamp,latitude,longitude,horizontal_accuracy,region,ping_near_replicate_matches,quarter,cluster,centroid_address,datetime,is_day,is_evening,is_night,date,hour
0,3924b2d36e1b036021dd5cc9ccabf33e20ba55e0f3a531...,90301,2024-02-14 05:33:33.000,33.95756,-118.37116,33.0,california,1,2,0,"{'latitude': 33.95752090909091, 'longitude': -...",2024-02-14 05:33:33,0,0,1,2024-02-14,5
1,3924b2d36e1b036021dd5cc9ccabf33e20ba55e0f3a531...,90301,2024-02-14 06:18:44.000,33.95456,-118.35538,5.0,california,1,2,1,"{'latitude': 33.954546, 'longitude': -118.3553...",2024-02-14 06:18:44,0,0,1,2024-02-14,6
2,3924b2d36e1b036021dd5cc9ccabf33e20ba55e0f3a531...,90301,2024-02-14 05:12:58.000,33.95747,-118.37079,12.0,california,1,2,0,"{'latitude': 33.95752090909091, 'longitude': -...",2024-02-14 05:12:58,0,0,1,2024-02-14,5
3,3924b2d36e1b036021dd5cc9ccabf33e20ba55e0f3a531...,90301,2024-02-14 05:33:38.000,33.95754,-118.37114,33.0,california,1,2,0,"{'latitude': 33.95752090909091, 'longitude': -...",2024-02-14 05:33:38,0,0,1,2024-02-14,5
4,3924b2d36e1b036021dd5cc9ccabf33e20ba55e0f3a531...,90301,2024-02-14 06:42:13.000,33.95672,-118.35954,5.0,california,1,2,2,"{'latitude': 33.956678, 'longitude': -118.3595...",2024-02-14 06:42:13,0,0,1,2024-02-14,6


# Create indicators (function)

In [ ]:
"""
This function creates all indicators mentioned in the indicators description
"""

import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import entropy
import time

def compute_bounded_streaks_with_silence(df, target_cluster):
    df = df.copy()
    df["hour_ts"] = df["datetime"].dt.floor("h")
    df = df.sort_values("hour_ts")

    cluster_hours = df[df["cluster"] == target_cluster]["hour_ts"].sort_values().to_list()
    hour_cluster_map = df.groupby("hour_ts")["cluster"].apply(set).to_dict()
    timeline = pd.date_range(df["hour_ts"].min(), df["hour_ts"].max(), freq="h")

    max_streak = 0
    current_start = None

    for hour in timeline:
        clusters = hour_cluster_map.get(hour, set())

        if not clusters:
            continue

        if clusters == {target_cluster}:
            if current_start is None:
                current_start = hour
        else:
            if current_start is not None:
                end_hour = max([h for h in cluster_hours if current_start <= h < hour], default=current_start)
                streak = int((end_hour - current_start) / pd.Timedelta(hours=1)) + 1
                max_streak = max(max_streak, streak)
                current_start = None

    if current_start is not None:
        end_hour = max([h for h in cluster_hours if h >= current_start])
        streak = int((end_hour - current_start) / pd.Timedelta(hours=1)) + 1
        max_streak = max(max_streak, streak)

    return max_streak

def compute_cluster_quarterly_metrics_vectorized_with_progress(final_data):
    df = final_data.copy()
    df["hour_ts"] = df["datetime"].dt.floor("h")

    print("🧱 [1/8] Computing corrected max streaks...")
    start = time.time()
    streak_results = []
    for (caid, quarter), group in tqdm(df.groupby(["caid", "quarter"]), desc="→ CAID/Quarter"):
        for cluster in group["cluster"].unique():
            max_streak = compute_bounded_streaks_with_silence(group, cluster)
            streak_results.append({
                "caid": caid,
                "quarter": quarter,
                "cluster": cluster,
                "max_consecutive_hours": max_streak
            })
    max_streaks = pd.DataFrame(streak_results)
    print(f"✅ Done in {time.time() - start:.2f}s")

    print("🧱 [2/8] Group-level aggregates...")
    base = df.groupby(['caid', 'quarter', 'cluster']).agg(
        total_pings=("datetime", "count"),
        unique_days=("date", "nunique"),
        unique_hours=("hour", "nunique"),
        zipcode=("zipcode", "first"),
        centroid_data=("centroid_address", "first"),
    ).reset_index()

    def extract_coord(d, key):
        try:
            d = eval(d) if isinstance(d, str) else d
            return d.get(key, np.nan)
        except:
            return np.nan

    base["centroid_latitude"] = base["centroid_data"].apply(lambda d: extract_coord(d, "latitude"))
    base["centroid_longitude"] = base["centroid_data"].apply(lambda d: extract_coord(d, "longitude"))
    base.drop(columns=["centroid_data"], inplace=True)

    print("🧱 [3/8] Log pings + general consistency...")
    base["log_total_pings"] = np.log1p(base["total_pings"])
    total_days = df.groupby(['caid', 'quarter'])['date'].nunique().rename("total_full_days_in_quarter").reset_index()
    base = base.merge(total_days, on=["caid", "quarter"], how="left")
    base["consistency_score"] = base["unique_days"] / base["total_full_days_in_quarter"]
    base.loc[base["total_full_days_in_quarter"].isna() | (base["total_full_days_in_quarter"] == 0), "consistency_score"] = np.nan

    print("🧱 [4/8] Time window consistency...")
    for period in ['day', 'evening', 'night']:
        is_period = f"is_{period}"
        period_days_col = f"total_{period}s_in_quarter"
        cluster_days_col = f"cluster_{period}_days"
        score_col = f"{period}_consistency_score"

        # Total period days per caid/quarter
        total_days = (
            df[df[is_period] == 1]
            .groupby(['caid', 'quarter'])['date']
            .nunique()
            .rename(period_days_col)
            .reset_index()
        )

        cluster_days = (
            df[df[is_period] == 1]
            .groupby(['caid', 'quarter', 'cluster'])['date']
            .nunique()
            .rename(cluster_days_col)
            .reset_index()
        )

        base = base.merge(total_days, on=["caid", "quarter"], how="left")
        base = base.merge(cluster_days, on=["caid", "quarter", "cluster"], how="left")

        # Fill NaN cluster days with 0
        base[cluster_days_col] = base[cluster_days_col].fillna(0)

        base[score_col] = base[cluster_days_col] / base[period_days_col]

        # Force NaN for all clusters if no opportunity
        base.loc[(base[period_days_col].isna()) | (base[period_days_col] == 0), score_col] = np.nan

    print("🧱 [5/8] Time window coverage...")
    flags = df.groupby(["caid", "quarter", "cluster"])[["is_day", "is_evening", "is_night"]].sum().gt(0).astype(int)
    flags["time_window_coverage"] = flags[["is_day", "is_evening", "is_night"]].sum(axis=1) / 3
    flags = flags[["time_window_coverage"]].reset_index()
    base = base.merge(flags, on=["caid", "quarter", "cluster"], how="left")

    print("🧱 [6/8] Weekend focus + dominance...")
    df["weekday"] = df["date"].dt.dayofweek

    weekend_total = df[df["weekday"] >= 5].groupby(["caid", "quarter"])["date"].nunique().rename("total_weekend_days").reset_index()
    weekday_total = df[df["weekday"] < 5].groupby(["caid", "quarter"])["date"].nunique().rename("total_weekday_days").reset_index()
    cluster_weekend = df[df["weekday"] >= 5].groupby(["caid", "quarter", "cluster"])["date"].nunique().rename("weekend_days").reset_index()
    cluster_weekday = df[df["weekday"] < 5].groupby(["caid", "quarter", "cluster"])["date"].nunique().rename("weekday_days").reset_index()

    base = base.merge(weekend_total, on=["caid", "quarter"], how="left")
    base = base.merge(weekday_total, on=["caid", "quarter"], how="left")
    base = base.merge(cluster_weekend, on=["caid", "quarter", "cluster"], how="left")
    base = base.merge(cluster_weekday, on=["caid", "quarter", "cluster"], how="left")

    base["weekend_focus_score"] = base["weekend_days"] / (base["weekend_days"] + base["weekday_days"] + 1e-5)

    no_opportunity = ((base["total_weekend_days"] + base["total_weekday_days"]).fillna(0) == 0)
    base.loc[no_opportunity, "weekend_focus_score"] = np.nan

    print("🧱 [7/8] Dominance score...")
    total_caid_pings = df.groupby(['caid', 'quarter'])['datetime'].count().rename("total_caid_pings").reset_index()
    base = base.merge(total_caid_pings, on=["caid", "quarter"], how="left")
    base["dominance_score"] = base["total_pings"] / base["total_caid_pings"]
    base.drop(columns=["total_caid_pings"], inplace=True)

    print("🧱 [8/8] Entropy + merge streaks...")
    entropy_df = df.groupby(['caid', 'quarter', 'cluster'])['hour'].apply(
        lambda s: entropy(s.value_counts(normalize=True)) if s.nunique() > 1 else np.nan
    ).reset_index(name="hour_entropy")
    base = base.merge(entropy_df, on=["caid", "quarter", "cluster"], how="left")
    base = base.merge(max_streaks, on=["caid", "quarter", "cluster"], how="left")

    print("✅ All done.")
    return base


# Calculate all indicators

In [62]:
cluster_quarterly_metrics = compute_cluster_quarterly_metrics_vectorized_with_progress(final_data)

🧱 [1/8] Computing corrected max streaks...


→ CAID/Quarter: 100%|██████████| 37986/37986 [51:24<00:00, 12.31it/s]   


✅ Done in 3088.67s
🧱 [2/8] Group-level aggregates...
🧱 [3/8] Log pings + general consistency...
🧱 [4/8] Time window consistency...
🧱 [5/8] Time window coverage...
🧱 [6/8] Weekend focus + dominance...
🧱 [7/8] Dominance score...
🧱 [8/8] Entropy + merge streaks...
✅ All done.


In [63]:
cluster_quarterly_metrics.to_csv("../datasets/cluster_quarterly_metrics.csv")

# Add census data

In [64]:
"""
We add median income household (2023) based on zip code, found in the census data.
"""
# Load census income data
census_data = pd.read_csv('../datasets/census_data.csv', skiprows=1)

# Extract ZIP code and rename income column
census_data['zipcode'] = census_data.iloc[:, 1].str.extract(r'(\d{5})')
census_data = census_data.rename(columns={census_data.columns[2]: 'median_income_household_2023'})
census_data = census_data[['zipcode', 'median_income_household_2023']]

# Ensure ZIPs are 5-digit strings
census_data['zipcode'] = census_data['zipcode'].astype(str).str.zfill(5)
cluster_quarterly_metrics['zipcode'] = cluster_quarterly_metrics['zipcode'].astype(str).str.zfill(5)

# Merge income into cluster-level metrics
cluster_quarterly_metrics = cluster_quarterly_metrics.merge(
    census_data,
    on='zipcode',
    how='left'
)

cluster_quarterly_metrics.head()


,caid,quarter,cluster,total_pings,unique_days,unique_hours,zipcode,centroid_latitude,centroid_longitude,log_total_pings,total_full_days_in_quarter,consistency_score,total_days_in_quarter,cluster_day_days,day_consistency_score,total_evenings_in_quarter,cluster_evening_days,evening_consistency_score,total_nights_in_quarter,cluster_night_days,night_consistency_score,time_window_coverage,total_weekend_days,total_weekday_days,weekend_days,weekday_days,weekend_focus_score,dominance_score,hour_entropy,max_consecutive_hours,median_income_household_2023
0,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,0,32,3,3,90020,34.065744,-118.296350,3.496508,4,0.750000,2.0,2.0,1.0,2.0,1.0,0.5,NaN,0.0,NaN,0.666667,1.0,3.0,NaN,3.0,NaN,0.666667,0.900256,253,55832
1,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,1,9,1,1,90002,33.959281,-118.253437,2.302585,4,0.250000,2.0,0.0,0.0,2.0,1.0,0.5,NaN,0.0,NaN,0.333333,1.0,3.0,1.0,NaN,NaN,0.187500,NaN,1,56158
2,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,2,7,1,1,91606,34.182663,-118.383647,2.079442,4,0.250000,2.0,0.0,0.0,2.0,1.0,0.5,NaN,0.0,NaN,0.333333,1.0,3.0,NaN,1.0,NaN,0.145833,NaN,1,66884
3,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,5,0,106,2,12,90020,34.065744,-118.296350,4.672829,3,0.666667,2.0,2.0,1.0,NaN,0.0,NaN,3.0,2.0,0.666667,0.666667,NaN,3.0,NaN,2.0,NaN,0.946429,2.257982,87,55832
4,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,5,3,6,1,2,90020,34.065623,-118.292500,1.945910,3,0.333333,2.0,0.0,0.0,NaN,0.0,NaN,3.0,1.0,0.333333,0.333333,NaN,3.0,NaN,1.0,NaN,0.053571,0.450561,4,55832


In [65]:
cluster_quarterly_metrics.shape

(1167343, 31)

In [66]:
# Observe results for first caid and quarter == 2
first_caid = cluster_quarterly_metrics['caid'].unique()[0]
cluster_quarterly_metrics[(cluster_quarterly_metrics['caid'] == first_caid) & (cluster_quarterly_metrics['quarter'] == 2)]

,caid,quarter,cluster,total_pings,unique_days,unique_hours,zipcode,centroid_latitude,centroid_longitude,log_total_pings,total_full_days_in_quarter,consistency_score,total_days_in_quarter,cluster_day_days,day_consistency_score,total_evenings_in_quarter,cluster_evening_days,evening_consistency_score,total_nights_in_quarter,cluster_night_days,night_consistency_score,time_window_coverage,total_weekend_days,total_weekday_days,weekend_days,weekday_days,weekend_focus_score,dominance_score,hour_entropy,max_consecutive_hours,median_income_household_2023
0,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,0,32,3,3,90020,34.065744,-118.296350,3.496508,4,0.75,2.0,2.0,1.0,2.0,1.0,0.5,NaN,0.0,NaN,0.666667,1.0,3.0,NaN,3.0,NaN,0.666667,0.900256,253,55832
1,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,1,9,1,1,90002,33.959281,-118.253437,2.302585,4,0.25,2.0,0.0,0.0,2.0,1.0,0.5,NaN,0.0,NaN,0.333333,1.0,3.0,1.0,NaN,NaN,0.187500,NaN,1,56158
2,000c2d116598ea942c398285b59f0e8ee465d200810bfa...,2,2,7,1,1,91606,34.182663,-118.383647,2.079442,4,0.25,2.0,0.0,0.0,2.0,1.0,0.5,NaN,0.0,NaN,0.333333,1.0,3.0,NaN,1.0,NaN,0.145833,NaN,1,66884


# Save data

In [67]:
cluster_quarterly_metrics.to_csv("../datasets/cluster_quarterly_metrics.csv")